In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from py2neo import Graph
import numpy as np

import matplotlib.pyplot as plt

# host + port
host = 'http://localhost:11008'

# select database name
db_name = 'modelparallel'

# credentials for API
user = 'neo4j'
passwd = '1234'

model_graph = Graph(host, auth=(user, passwd), name=db_name)

In [42]:
tendencies = model_graph.run("""
        MATCH (m1:Molecule)-[s:SAME_AS]->(m2:Molecule)
        RETURN m1.formula_string AS from_formula, m1.measurement_id AS from_mid, m2.formula_string AS to_formula, m2.measurement_id AS to_mid, s.intensity_trend as intensity_trend, m1.peak_relint_tic as int
        ORDER BY intensity_trend ASC
""").to_data_frame()

print(tendencies)

print(len(tendencies))

tendencies_lower = tendencies[tendencies['intensity_trend'] <= 0.975]
tendencies_upper = tendencies[tendencies['intensity_trend'] >= 1.025]

tendencies_equal = tendencies[tendencies['intensity_trend'] < 1.025]
tendencies_equal = tendencies_equal[tendencies_equal['intensity_trend'] > 0.975]

print(len(tendencies_lower))
print(len(tendencies_upper))
print(len(tendencies_equal))
print(len(tendencies_lower)+len(tendencies_upper)+len(tendencies_equal))

        from_formula from_mid     to_formula to_mid  intensity_trend       int
0      C22 H41 N1 O3     7815  C22 H41 N1 O3   7821            0.085  0.000433
1      C22 H43 N1 O1     7815  C22 H43 N1 O1   7821            0.093  0.001337
2      C22 H43 N1 O3     7815  C22 H43 N1 O3   7821            0.095  0.000598
3      C22 H41 N1 O2     7815  C22 H41 N1 O2   7821            0.101  0.000822
4      C22 H39 N1 O3     7815  C22 H39 N1 O3   7821            0.105  0.000489
...              ...      ...            ...    ...              ...       ...
79944    C6 H8 N4 S3     7821    C6 H8 N4 S3   7827            4.112  0.000035
79945    C5 H6 N4 S3     7821    C5 H6 N4 S3   7827            4.170  0.000022
79946  C22 H39 N1 O3     7827  C22 H39 N1 O3   7833            4.549  0.000027
79947     C14 H22 O1     7851     C14 H22 O1   7857            4.661  0.000085
79948  C18 H35 N1 O7     7857  C18 H35 N1 O7   7863            4.786  0.000042

[79949 rows x 6 columns]
79949
38922
30258
10769
79

In [44]:
# increasing intensity_trend

upper_limit = 1.025
percent = 1.025
percent_factor = 0.025
weight = 0.02
increase_weight = 0.0103
border = upper_limit*percent
upper_weight_list = []

for row in tendencies_upper.itertuples():
    if row.intensity_trend < border:
        upper_weight_list.append(weight*(row.int))
    else:
        percent = percent + percent_factor
        weight = weight + increase_weight
        border = upper_limit * percent        
        upper_weight_list.append(weight*(row.int))
        
    #print(row.intensity_trend, border, weight)
    
print(len(upper_weight_list))
tendencies_upper['tendency_weight'] = upper_weight_list
tendencies_upper

30258


<ipython-input-44-390ccecfd6a1>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tendencies_upper['tendency_weight'] = upper_weight_list


,from_formula,from_mid,to_formula,to_mid,intensity_trend,int,tendency_weight
49691,C16 H33 N5 O3 S2,7863,C16 H33 N5 O3 S2,7868,1.025,0.000144,2.874660e-06
49692,C15 H14 O3,7851,C15 H14 O3,7857,1.025,0.000132,2.632700e-06
49693,C15 H17 N1 O10,7863,C15 H17 N1 O10,7868,1.025,0.000048,9.580000e-07
49694,C15 H17 N1 O7,7857,C15 H17 N1 O7,7863,1.025,0.000225,4.498280e-06
49695,C15 H18 N2 O6,7868,C15 H18 N2 O6,7874,1.025,0.000144,2.882180e-06
...,...,...,...,...,...,...,...
79944,C6 H8 N4 S3,7821,C6 H8 N4 S3,7827,4.112,0.000035,3.340977e-05
79945,C5 H6 N4 S3,7821,C5 H6 N4 S3,7827,4.170,0.000022,2.157748e-05
79946,C22 H39 N1 O3,7827,C22 H39 N1 O3,7833,4.549,0.000027,2.620772e-05
79947,C14 H22 O1,7851,C14 H22 O1,7857,4.661,0.000085,8.379936e-05


In [39]:
# decreasing intensity_trend

tendencies_lower = tendencies_lower.sort_values(by='intensity_trend', ascending=False)

lower_limit = 0.975
percent = 0.975 
percent_factor = 0.025
weight = 0.02
increase_weight = 0.0272
border = lower_limit*percent
lower_weight_list = []

for row in tendencies_lower.itertuples():
    if border < row.intensity_trend:
        lower_weight_list.append(weight*row.int)
    else:
        percent = percent - percent_factor
        weight = weight + increase_weight
        border = lower_limit * percent        
        lower_weight_list.append(weight*row.int)
        
    #print(row.intensity_trend, border, weight)
    
print(len(lower_weight_list))
tendencies_lower['tendency_weight'] = lower_weight_list
tendencies_lower

38922


,from_formula,from_mid,to_formula,to_mid,intensity_trend,int,tendency_weight
38921,C19 H28 N2 O9 S3,7827,C19 H28 N2 O9 S3,7833,0.975,0.000094,1.886000e-06
38764,C13 H14 O5,7845,C13 H14 O5,7851,0.975,0.001143,2.286210e-05
38747,C16 H26 O2,7821,C16 H26 O2,7827,0.975,0.000032,6.440000e-07
38792,C17 H22 O10,7821,C17 H22 O10,7827,0.975,0.000738,1.475870e-05
38804,C12 H11 N1 O4,7833,C12 H11 N1 O4,7839,0.975,0.000098,1.968000e-06
...,...,...,...,...,...,...,...
4,C22 H39 N1 O3,7815,C22 H39 N1 O3,7821,0.105,0.000489,4.748366e-04
3,C22 H41 N1 O2,7815,C22 H41 N1 O2,7821,0.101,0.000822,7.985787e-04
2,C22 H43 N1 O3,7815,C22 H43 N1 O3,7821,0.095,0.000598,5.978753e-04
1,C22 H43 N1 O1,7815,C22 H43 N1 O1,7821,0.093,0.001337,1.335934e-03


In [40]:
equal_weight_list = []

for row in tendencies_equal.itertuples():
    equal_weight_list.append(0.0)

print(len(equal_weight_list))
tendencies_equal['tendency_weight'] = equal_weight_list
tendencies_equal

10769


,from_formula,from_mid,to_formula,to_mid,intensity_trend,int,tendency_weight
38922,C19 H24 O7,7857,C19 H24 O7,7863,0.976,0.001734,0.0
38923,C19 H24 O9 S1,7868,C19 H24 O9 S1,7874,0.976,0.000033,0.0
38924,C19 H25 N1 O6,7803,C19 H25 N1 O6,7809,0.976,0.000103,0.0
38925,C19 H26 O5,7863,C19 H26 O5,7868,0.976,0.000721,0.0
38926,C19 H26 O7,7815,C19 H26 O7,7821,0.976,0.002096,0.0
...,...,...,...,...,...,...,...
49686,C17 H22 O7,7815,C17 H22 O7,7821,1.024,0.002353,0.0
49687,C15 H24 O6 S1,7851,C15 H24 O6 S1,7857,1.024,0.000116,0.0
49688,C15 H27 N5 O6 S2,7809,C15 H27 N5 O6 S2,7815,1.024,0.000137,0.0
49689,C15 H27 N5 O7 S2,7868,C15 H27 N5 O7 S2,7874,1.024,0.000098,0.0


In [10]:
frames = [tendencies_upper, tendencies_lower, tendencies_equal]
concat_df = pd.concat(frames)
concat_df.to_csv('test.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
MATCH (m:Molecule)-[t:HAS_TRANSFORMED_INTO]->(:Molecule)
WITH m.formula_string as fs, m.measurement_id as mid, sum(t.combined_tendency_weight) as sum_weight
MATCH (m1:Molecule)-[t1:HAS_TRANSFORMED_INTO]->(:Molecule)
WHERE m1.formula_string = fs AND m1.measurement_id = mid
SET t1.normalized_combined_weight = apoc.math.round(t1.combined_tendency_weight/sum_weight, 3)
RETURN fs, mid, sum_weight